# VacationPy: Plottling Geo-Data of weather cities.

## Objectives
This notebook plots geo-location of cities randomly generated in new_weatherpy.

## Import dependencies

In [182]:
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import gmaps
import matplotlib.pyplot as plt
from config import gkey
from config import api_key, API_KEY

**Import weather data: data_df**

In [183]:
# Import weather stats
data_df = pd.read_csv('weather_stats.csv')
data_df.head()

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,-22.97,-42.03,Arraial do Cabo,BR,297.15,88,6.20,2,1601509997
1,-33.93,18.42,Cape Town,ZA,286.48,81,10.80,75,1601509880
2,-26.65,15.16,Lüderitz,NaN,285.45,68,7.89,5,1601509997
3,5.89,95.32,Sabang,ID,300.80,72,1.02,23,1601509998
4,38.89,-76.51,Mayo,US,291.15,63,2.10,1,1601509998


- Convert temperature from Kelvin to fahrenheit.

In [184]:
# Convert Max Temp to fahrenheit
deg_f = [(9/5)*(t - 273) + 32 for t in data_df['Max Temp']]
data_df['Max Temp'] = deg_f
data_df.head()

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,-22.97,-42.03,Arraial do Cabo,BR,75.470,88,6.20,2,1601509997
1,-33.93,18.42,Cape Town,ZA,56.264,81,10.80,75,1601509880
2,-26.65,15.16,Lüderitz,NaN,54.410,68,7.89,5,1601509997
3,5.89,95.32,Sabang,ID,82.040,72,1.02,23,1601509998
4,38.89,-76.51,Mayo,US,64.670,63,2.10,1,1601509998


### Weather Cities Map

- Configure gmaps with API key

In [172]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

- Store 'Latitude' and 'Longitude' into  locations.
- Drop all NaN values.
- Convert Humidity to float

In [173]:
# Store 'Lat' and 'Lng' into  locations 
locations = data_df[["Latitude", "Longitude"]].astype(float)

# Convert Humidity to float and store
# Handle NaN values
data_df = data_df.dropna()
humidity = data_df["Humidity"].astype(float)


- Create a Humidity Heatmap layer

In [174]:
# Create a Humidity Heatmap layer


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=75,
                                 point_radius =5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Ideal Weather conditions
- Extract all weather data '**data_ideal**' that fulfils the following conditions:
  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

In [185]:
data_ideal = data_df.loc[((data_df['Max Temp'] <= 80) & (data_df['Max Temp'] >= 70)) & (data_df['Wind Speed'] < 10) & \
                        (data_df['Cloudiness']==0)]
data_ideal

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
42,31.76,13.99,Bani Walid,LY,70.304,81,2.60,0,1601510012
65,30.33,10.85,Nālūt,LY,75.704,24,3.37,0,1601510020
212,31.61,34.76,Kiryat Gat,IL,76.262,83,0.39,0,1601510075
213,29.82,31.05,Madīnat Sittah Uktūbar,EG,79.070,83,3.60,0,1601510076
222,-11.38,43.28,Mitsamiouli,KM,78.494,77,2.62,0,1601510079
261,32.12,20.07,Benghazi,LY,73.688,66,4.20,0,1601510093
347,29.03,21.55,Jalu,LY,77.396,45,5.18,0,1601510125
356,-23.70,133.88,Alice Springs,AU,75.470,22,6.70,0,1601510064
367,23.67,-100.18,Doctor Arroyo,MX,71.204,33,1.61,0,1601510132
374,-21.64,-41.05,São João da Barra,BR,74.912,84,9.18,0,1601510135


In [186]:
# Convert Max Temp to fahrenheit
deg_f = [(9/5)*(t - 273) + 32 for t in data_df['Max Temp']]
data_df['Max Temp'] = deg_f
data_df.head()

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,-22.97,-42.03,Arraial do Cabo,BR,-323.5540,88,6.20,2,1601509997
1,-33.93,18.42,Cape Town,ZA,-358.1248,81,10.80,75,1601509880
2,-26.65,15.16,Lüderitz,NaN,-361.4620,68,7.89,5,1601509997
3,5.89,95.32,Sabang,ID,-311.7280,72,1.02,23,1601509998
4,38.89,-76.51,Mayo,US,-342.9940,63,2.10,1,1601509998


**Generate hotels via API calls to OpenweatherMap API**

In [61]:
hotel_name = []

# Nearest Hotel for each city within 5000 meters

hotel_loc = data_ideal[['Latitude', 'Longitude', 'City', 'Country']]

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "type": "hotel",
    "radius": 5000,
    "key": gkey,
}


count = 1

for index, row in hotel_loc.iterrows():
    
    lat = row['Latitude']
    lon = row['Longitude']

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
     # assemble url and make API request
    print(f"Retrieving data of hotel: {count}")
    print(f"..................................")
    count = count + 1

    # make request 
    hotel_response = requests.get(base_url, params=params)
    
    hotel_json = hotel_response.json()
    
    try:

        
        hotel_name.append(hotel_json['results'][0]['name'])
        
    
    except (KeyError, IndexError):
        
        hotel_name.append(np.nan)
        print("Hotel not found... skipping.")
        
    print("-----------------")

Retrieving data of hotel: 1
..................................
-----------------
Retrieving data of hotel: 2
..................................
Hotel not found... skipping.
-----------------
Retrieving data of hotel: 3
..................................
Hotel not found... skipping.
-----------------
Retrieving data of hotel: 4
..................................
Hotel not found... skipping.
-----------------
Retrieving data of hotel: 5
..................................
-----------------
Retrieving data of hotel: 6
..................................
-----------------
Retrieving data of hotel: 7
..................................
-----------------
Retrieving data of hotel: 8
..................................
-----------------
Retrieving data of hotel: 9
..................................
-----------------
Retrieving data of hotel: 10
..................................
-----------------
Retrieving data of hotel: 11
..................................
-----------------
Retrieving data of h

- Save Hotel information in a new column of hotel_loc: **Hotel**

In [62]:
hotel_loc['Hotel']= hotel_name

C:\Users\lenpk\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- Save hotel data in csv file: **hotels.csv**

In [129]:
hotel_loc.to_csv('hotels.csv', index = False)

- Load hotels data in a data frame: **hotels**

In [130]:
# Load Hotels data
hotels = pd.read_csv('hotels.csv')
hotels.head()

,Latitude,Longitude,City,Country,Hotel
0,31.76,13.99,Bani Walid,LY,Zaitouna Hotel
1,-11.38,43.28,Mitsamiouli,KM,Al Camar Lodge
2,32.12,20.07,Benghazi,LY,Ishbiliyah Beach Hotel
3,29.03,21.55,Jalu,LY,Jalu Hotel
4,-23.70,133.88,Alice Springs,AU,DoubleTree by Hilton Hotel Alice Springs


- Replace Arabic name to translated English name

In [131]:
hotels['Hotel'].replace({'فندق الزيتونة':'Zaitouna Hotel'}, inplace = True)
hotels

,Latitude,Longitude,City,Country,Hotel
0,31.76,13.99,Bani Walid,LY,Zaitouna Hotel
1,-11.38,43.28,Mitsamiouli,KM,Al Camar Lodge
2,32.12,20.07,Benghazi,LY,Ishbiliyah Beach Hotel
3,29.03,21.55,Jalu,LY,Jalu Hotel
4,-23.70,133.88,Alice Springs,AU,DoubleTree by Hilton Hotel Alice Springs
5,23.67,-100.18,Doctor Arroyo,MX,HOTEL LOS PINOS
6,-21.64,-41.05,São João da Barra,BR,Pousada Porto Prime
7,26.33,43.98,Buraidah,SA,Mövenpick Hotel Qassim
8,20.93,-17.03,Nouadhibou,MR,Free Zone Hotel Nouadhibou
9,-15.86,27.75,Mazabuka,ZM,Royal Laricio


- Drop all NaN values
- Reset index

In [132]:
hotels.dropna(subset=['Hotel'], inplace = True)
hotels.reset_index(drop = True, inplace = True)
hotels

,Latitude,Longitude,City,Country,Hotel
0,31.76,13.99,Bani Walid,LY,Zaitouna Hotel
1,-11.38,43.28,Mitsamiouli,KM,Al Camar Lodge
2,32.12,20.07,Benghazi,LY,Ishbiliyah Beach Hotel
3,29.03,21.55,Jalu,LY,Jalu Hotel
4,-23.70,133.88,Alice Springs,AU,DoubleTree by Hilton Hotel Alice Springs
5,23.67,-100.18,Doctor Arroyo,MX,HOTEL LOS PINOS
6,-21.64,-41.05,São João da Barra,BR,Pousada Porto Prime
7,26.33,43.98,Buraidah,SA,Mövenpick Hotel Qassim
8,20.93,-17.03,Nouadhibou,MR,Free Zone Hotel Nouadhibou
9,-15.86,27.75,Mazabuka,ZM,Royal Laricio


**Plot Map of hotels**

- Create info-box-template
- Extract locations points: **locations**
- Add markers

In [133]:
# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotels = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotel_loc[["Latitude", "Longitude"]]

In [77]:
# Add Markers
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))